In [58]:
import os
import pickle
from labels import *
from utils import read_json_to_dict, read_and_set_keywords


In [1]:
import copy
import collections

from PetReader import pet_reader
from petreader.labels import *

INFO:PetReader:Load RelationsExtraction dataset ...




 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]



  0%|          | 0/1 [00:00<?, ?it/s]

INFO:PetReader:Load TokenClassification dataset ...




 _______ _     _ _______       _____  _______ _______      ______  _______ _______ _______ _______ _______ _______
    |    |_____| |______      |_____] |______    |         |     \ |_____|    |    |_____| |______ |______    |   
    |    |     | |______      |       |______    |         |_____/ |     |    |    |     | ______| |______    |   
                                                                                                                  
Discover more at: [https://pdi.fbk.eu/pet-dataset/]



  0%|          | 0/1 [00:00<?, ?it/s]

## Load Results

In [6]:
token_cls_goldstandard = read_json_to_dict("data/other/token_goldstandard.json")

keywords_gold_token_cls_results = read_json_to_dict("data/results/key_words_gold/results-token-classification.json")
keywords_gold_token_cls = read_json_to_dict("data/results/key_words_gold/token-classification.json")

keywords_literature_token_cls_results = read_json_to_dict("data/results/key_words_literature/results-token-classification.json")
keywords_literature_token_cls = read_json_to_dict("data/results/key_words_literature/token-classification.json")

## Debug

In [3]:
from petreader.TokenClassification import TokenClassification
from petbenchmarks.tokenclassification import TokenClassificationBenchmark
benchmark = TokenClassificationBenchmark(pet_reader.token_dataset)

gold = benchmark.GetGoldStandard()
print(gold['doc-3.2'][XOR_GATEWAY])

['If', 'otherwise']


In [4]:
print(pet_reader.token_dataset.GetXORGateways('doc-3.2'))

[[], [['If'], ['otherwise']], [], []]


## 1) Analyze Keyword Results

### a) check for documents with recall != 1 (GOLD)

In [5]:
for doc_name, doc_dict in keywords_gold_token_cls_results.items():
    if doc_name.startswith('doc'):
        if doc_dict[XOR_GATEWAY][RECALL] != 1 and doc_dict[XOR_GATEWAY][SUPPORT] != 0:
            print(doc_name.center(100, '-'))
            print("--Text--")
            for i, line in enumerate(pet_reader.get_doc_sentences(doc_name)):
                print(i, ' '.join(line))
            print()
            print("--Results--:", doc_dict[XOR_GATEWAY])
            print()
            print("--Extracted--:", keywords_gold_token_cls[doc_name][XOR_GATEWAY])
            print()
            print("--Gold standard--:", token_cls_goldstandard[doc_name][XOR_GATEWAY])
            print()

### b) In how many different documents appear the Gateways?

In [61]:
goldstandard_object_path = "data/other/token_goldstandard.pkl"
with open(os.path.abspath(goldstandard_object_path), 'rb') as file:
    goldstandard = pickle.load(file)

In [99]:
def get_gateway_stats(classifications, gateway_type):
    gateway_stats = {}
    for doc_name, doc_dict in classifications.items():
        for gateway_token in doc_dict[gateway_type]:
            if gateway_token not in gateway_stats:
                gateway_stats[gateway_token] = {"count": 1, "docs": 1, "doc_list": set([doc_name])}
            else:
                gateway_stats[gateway_token]["count"] += 1
                gateway_stats[gateway_token]["doc_list"].add(doc_name)
                gateway_stats[gateway_token]["docs"] = len(gateway_stats[gateway_token]["doc_list"])
            
    gateway_stats = collections.OrderedDict(sorted(gateway_stats.items(), 
                                                   key=lambda kv_pair: (kv_pair[1]["count"], kv_pair[1]["docs"]),
                                                   reverse=True))
    return gateway_stats

In [113]:
# XOR GOLDSTANDARD
goldstandard_xor_gateway_stats = get_gateway_stats(goldstandard, XOR_GATEWAY)
for gateway, stats in gold_xor_gateway_stats.items():
    print(gateway.ljust(15), str(stats)[:100])

or              {'count': 64, 'docs': 29, 'doc_list': {'doc-10.14', 'doc-10.2', 'doc-6.1', 'doc-5.2', 'doc-10.12', '
for             {'count': 59, 'docs': 23, 'doc_list': {'doc-8.2', 'doc-10.14', 'doc-8.3', 'doc-1.4', 'doc-6.1', 'doc
If              {'count': 55, 'docs': 24, 'doc_list': {'doc-8.2', 'doc-10.14', 'doc-10.2', 'doc-6.1', 'doc-5.2', 'do
case            {'count': 15, 'docs': 4, 'doc_list': {'doc-2.1', 'doc-9.5', 'doc-2.2', 'doc-6.4'}}
if              {'count': 12, 'docs': 10, 'doc_list': {'doc-1.3', 'doc-9.5', 'doc-10.2', 'doc-6.3', 'doc-6.4', 'doc-
For             {'count': 10, 'docs': 4, 'doc_list': {'doc-5.4', 'doc-4.1', 'doc-2.2', 'doc-2.1'}}
In              {'count': 9, 'docs': 3, 'doc_list': {'doc-9.5', 'doc-2.2', 'doc-2.1'}}
of              {'count': 8, 'docs': 3, 'doc_list': {'doc-9.5', 'doc-2.2', 'doc-6.4'}}
otherwise       {'count': 7, 'docs': 5, 'doc_list': {'doc-3.2', 'doc-1.2', 'doc-8.3', 'doc-2.1', 'doc-9.1'}}
Otherwise       {'count': 6, 'docs': 5, 'doc_list':

In [114]:
# XOR GOLD
gold_xor_gateway_stats = get_gateway_stats(keywords_gold_token_cls, XOR_GATEWAY)
for gateway, stats in gold_xor_gateway_stats.items():
    print(gateway.ljust(15), str(stats)[:100])

or              {'count': 64, 'docs': 29, 'doc_list': {'doc-10.14', 'doc-10.2', 'doc-6.1', 'doc-5.2', 'doc-10.12', '
for             {'count': 59, 'docs': 23, 'doc_list': {'doc-8.2', 'doc-10.14', 'doc-8.3', 'doc-1.4', 'doc-6.1', 'doc
If              {'count': 55, 'docs': 24, 'doc_list': {'doc-8.2', 'doc-10.14', 'doc-10.2', 'doc-6.1', 'doc-5.2', 'do
case            {'count': 15, 'docs': 4, 'doc_list': {'doc-2.1', 'doc-9.5', 'doc-2.2', 'doc-6.4'}}
if              {'count': 12, 'docs': 10, 'doc_list': {'doc-1.3', 'doc-9.5', 'doc-10.2', 'doc-6.3', 'doc-6.4', 'doc-
For             {'count': 10, 'docs': 4, 'doc_list': {'doc-5.4', 'doc-4.1', 'doc-2.2', 'doc-2.1'}}
In              {'count': 9, 'docs': 3, 'doc_list': {'doc-9.5', 'doc-2.2', 'doc-2.1'}}
of              {'count': 8, 'docs': 3, 'doc_list': {'doc-9.5', 'doc-2.2', 'doc-6.4'}}
otherwise       {'count': 7, 'docs': 5, 'doc_list': {'doc-3.2', 'doc-1.2', 'doc-8.3', 'doc-2.1', 'doc-9.1'}}
Otherwise       {'count': 6, 'docs': 5, 'doc_list':

In [119]:
# XOR LITERATURE
literature_xor_gateway_stats = get_gateway_stats(keywords_literature_token_cls, XOR_GATEWAY)
for gateway, stats in literature_xor_gateway_stats.items():
    print(gateway.ljust(15), str(stats)[:100])

or              {'count': 64, 'docs': 29, 'doc_list': {'doc-10.14', 'doc-10.2', 'doc-6.1', 'doc-5.2', 'doc-10.12', '
If              {'count': 55, 'docs': 24, 'doc_list': {'doc-8.2', 'doc-10.14', 'doc-10.2', 'doc-6.1', 'doc-5.2', 'do
whether         {'count': 13, 'docs': 9, 'doc_list': {'doc-10.12', 'doc-3.6', 'doc-3.8', 'doc-6.3', 'doc-2.2', 'doc-
if              {'count': 12, 'docs': 10, 'doc_list': {'doc-1.3', 'doc-9.5', 'doc-10.2', 'doc-6.3', 'doc-2.1', 'doc-
case            {'count': 11, 'docs': 4, 'doc_list': {'doc-6.4', 'doc-9.5', 'doc-2.2', 'doc-2.1'}}
when            {'count': 8, 'docs': 7, 'doc_list': {'doc-5.3', 'doc-6.2', 'doc-3.8', 'doc-1.4', 'doc-8.1', 'doc-5.2
In              {'count': 8, 'docs': 3, 'doc_list': {'doc-9.5', 'doc-2.2', 'doc-2.1'}}
When            {'count': 7, 'docs': 7, 'doc_list': {'doc-8.2', 'doc-1.3', 'doc-3.6', 'doc-8.3', 'doc-3.5', 'doc-6.1
otherwise       {'count': 7, 'docs': 5, 'doc_list': {'doc-3.2', 'doc-1.2', 'doc-8.3', 'doc-2.1', 'doc-9.1'}}
of 

In [115]:
# AND GOLDSTANDARD
goldstandard_and_gateway_stats = get_gateway_stats(goldstandard, AND_GATEWAY)
for gateway, stats in goldstandard_and_gateway_stats.items():
    print(gateway.ljust(15), str(stats)[:100])

the             {'count': 2, 'docs': 2, 'doc_list': {'doc-1.1', 'doc-2.2'}}
meantime        {'count': 2, 'docs': 2, 'doc_list': {'doc-1.1', 'doc-3.2'}}
While           {'count': 2, 'docs': 2, 'doc_list': {'doc-1.3', 'doc-1.4'}}
At              {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
same            {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
time            {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
In              {'count': 1, 'docs': 1, 'doc_list': {'doc-1.1'}}
two             {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
concurrent      {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
activities      {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
are             {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
triggered       {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
Meantime        {'count': 1, 'docs': 1, 'doc_list': {'doc-3.5'}}
whereas         {'count': 1, 'docs': 1, 'doc_list': {'doc-1.2'}}


In [116]:
# AND GOLD
gold_and_gateway_stats = get_gateway_stats(keywords_gold_token_cls, AND_GATEWAY)
for gateway, stats in gold_and_gateway_stats.items():
    print(gateway.ljust(15), str(stats)[:100])

whereas         {'count': 2, 'docs': 2, 'doc_list': {'doc-9.5', 'doc-1.2'}}
While           {'count': 2, 'docs': 2, 'doc_list': {'doc-1.3', 'doc-1.4'}}
the             {'count': 2, 'docs': 2, 'doc_list': {'doc-1.1', 'doc-2.2'}}
meantime        {'count': 2, 'docs': 2, 'doc_list': {'doc-1.1', 'doc-3.2'}}
Meantime        {'count': 1, 'docs': 1, 'doc_list': {'doc-3.5'}}
At              {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
same            {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
time            {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
In              {'count': 1, 'docs': 1, 'doc_list': {'doc-1.1'}}
two             {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
concurrent      {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
activities      {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
are             {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}
triggered       {'count': 1, 'docs': 1, 'doc_list': {'doc-2.1'}}


In [120]:
# AND LITERATURE
literature_and_gateway_stats = get_gateway_stats(keywords_literature_token_cls, AND_GATEWAY)
for gateway, stats in literature_and_gateway_stats.items():
    print(gateway.ljust(15), str(stats)[:100])

whereas         {'count': 2, 'docs': 2, 'doc_list': {'doc-9.5', 'doc-1.2'}}
While           {'count': 2, 'docs': 2, 'doc_list': {'doc-1.3', 'doc-1.4'}}
In              {'count': 2, 'docs': 2, 'doc_list': {'doc-1.1', 'doc-1.4'}}
the             {'count': 2, 'docs': 2, 'doc_list': {'doc-1.1', 'doc-2.2'}}
meantime        {'count': 2, 'docs': 2, 'doc_list': {'doc-1.1', 'doc-3.2'}}
addition        {'count': 2, 'docs': 1, 'doc_list': {'doc-1.4'}}
to              {'count': 2, 'docs': 1, 'doc_list': {'doc-1.4'}}
Meantime        {'count': 1, 'docs': 1, 'doc_list': {'doc-3.5'}}
in              {'count': 1, 'docs': 1, 'doc_list': {'doc-1.4'}}
At              {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
same            {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}
time            {'count': 1, 'docs': 1, 'doc_list': {'doc-2.2'}}


### c) Analyze False Positives

In [140]:
def analyze_keyword_metrics(gold_standard, extracted_tokens, gateway_type: str, doc_names=[]):
    keyword_metrics = {}
    empty_stats_dict = {"TP": 0, "FP": 0, "FN": 0, "TPs": [], "FPs": [], "FNs": []}

    for doc_name, doc_dict in keywords_gold_token_cls_results.items():
        
        if doc_name.startswith('doc'):  # result dict contains as well keys for statistics            
            if not doc_names or doc_name in doc_names:  # filter for (optionally) passed list

                # remove gateways step by step if they were identified; left overs are false positives
                not_discovered_golds = [g.lower() for g in gold_standard[doc_name][gateway_type].copy()]

                for extracted in extracted_tokens[doc_name][gateway_type]:
                    extracted = extracted.lower()

                    # setup keyword dict in case it was not observed yet
                    if extracted not in keyword_metrics:
                        keyword_metrics[extracted] = copy.deepcopy(empty_stats_dict)

                    # 1) CHECK FOR FPs
                    if extracted not in not_discovered_golds:
                        keyword_metrics[extracted]["FP"] += 1
                        keyword_metrics[extracted]["FPs"].append(doc_name)

                    # 2) CHECK FOR TPs
                    else:
                        keyword_metrics[extracted]["TP"] += 1
                        keyword_metrics[extracted]["TPs"].append(doc_name)
                        not_discovered_golds.remove(extracted)

                # 3) FILL FNs (FPs from list not_discovered_xor_golds got removed during previous loop)
                for not_extracted in not_discovered_golds:

                    # setup keyword dict in case it was not observed yet
                    if not_extracted not in keyword_metrics:
                        keyword_metrics[not_extracted] = copy.deepcopy(empty_stats_dict)

                    keyword_metrics[not_extracted]["FN"] += 1
                    keyword_metrics[not_extracted]["FNs"].append(doc_name)
    return keyword_metrics


def print_keyword_dict(keyword_metrics, first_x=None, order_by=None):
    
    if order_by:
        keyword_metrics = collections.OrderedDict(sorted(keyword_metrics.items(), key=lambda kv_pair: kv_pair[1][order_by],
                                                         reverse=True))
    for keyword, metrics in keyword_metrics.items():
        print(f"{keyword} ".ljust(15), end='')
        for k, v in metrics.items():
            if len(k) == 3 and k.endswith("s"):
                if first_x:
                    print(f"{k}: ", end='')
                    print(v[:first_x], "...", end='')
                    print(", ", end='')
            else:
                print(f"{k}: ", end='')
                print(v, end='')
                print(", ", end='')
        print()


gold_xor_keyword_metrics = analyze_keyword_stats(token_cls_goldstandard, keywords_gold_token_cls, XOR_GATEWAY)
gold_and_keyword_metrics = analyze_keyword_stats(token_cls_goldstandard, keywords_gold_token_cls, AND_GATEWAY)
literature_xor_keyword_metrics = analyze_keyword_stats(token_cls_goldstandard, keywords_literature_token_cls, XOR_GATEWAY)
literature_and_keyword_metrics = analyze_keyword_stats(token_cls_goldstandard, keywords_literature_token_cls, AND_GATEWAY)

#### EXCLUSIVE

In [108]:
# GOLD keywords
print_keyword_dict(gold_xor_keyword_metrics, first_x=2, order_by="FP")

for            TP: 6, FP: 63, FN: 0, TPs: ['doc-2.1', 'doc-2.1'] ..., FPs: ['doc-3.6', 'doc-2.2'] ..., FNs: [] ..., 
or             TP: 20, FP: 44, FN: 0, TPs: ['doc-2.2', 'doc-1.1'] ..., FPs: ['doc-3.6', 'doc-2.2'] ..., FNs: [] ..., 
if             TP: 61, FP: 6, FN: 0, TPs: ['doc-3.6', 'doc-3.6'] ..., FPs: ['doc-2.1', 'doc-10.10'] ..., FNs: [] ..., 
should         TP: 2, FP: 6, FN: 0, TPs: ['doc-6.1', 'doc-6.1'] ..., FPs: ['doc-2.2', 'doc-2.2'] ..., FNs: [] ..., 
either         TP: 1, FP: 4, FN: 0, TPs: ['doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.1'] ..., FNs: [] ..., 
in             TP: 9, FP: 3, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.2'] ..., FNs: [] ..., 
case           TP: 12, FP: 3, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.2'] ..., FNs: [] ..., 
of             TP: 5, FP: 3, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.2'] ..., FNs: [] ..., 
otherwise      TP: 12, FP: 1, FN: 0, TPs: ['doc-3.6', 'doc-3.6'] ..., FPs: [

In [109]:
# LITERATURE keywords
print_keyword_dict(literature_xor_keyword_metrics, first_x=2, order_by="FP")

or             TP: 20, FP: 44, FN: 0, TPs: ['doc-2.2', 'doc-1.1'] ..., FPs: ['doc-3.6', 'doc-2.2'] ..., FNs: [] ..., 
when           TP: 0, FP: 15, FN: 0, TPs: [] ..., FPs: ['doc-3.6', 'doc-6.2'] ..., FNs: [] ..., 
whether        TP: 0, FP: 13, FN: 0, TPs: [] ..., FPs: ['doc-3.6', 'doc-2.2'] ..., FNs: [] ..., 
if             TP: 61, FP: 6, FN: 0, TPs: ['doc-3.6', 'doc-3.6'] ..., FPs: ['doc-2.1', 'doc-10.10'] ..., FNs: [] ..., 
either         TP: 1, FP: 4, FN: 0, TPs: ['doc-2.2'] ..., FPs: ['doc-2.2', 'doc-2.1'] ..., FNs: [] ..., 
in             TP: 9, FP: 2, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-6.4'] ..., FNs: [] ..., 
case           TP: 9, FP: 2, FN: 3, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-6.4'] ..., FNs: ['doc-2.1', 'doc-2.1'] ..., 
of             TP: 5, FP: 2, FN: 0, TPs: ['doc-2.2', 'doc-2.2'] ..., FPs: ['doc-2.2', 'doc-6.4'] ..., FNs: [] ..., 
only           TP: 0, FP: 2, FN: 0, TPs: [] ..., FPs: ['doc-2.2', 'doc-4.1'] ..., FNs: [] ..., 


#### PARALLEL

In [110]:
# GOLD keywords
print_keyword_dict(gold_and_keyword_metrics, first_x=2, order_by="FP")

whereas        TP: 1, FP: 1, FN: 0, TPs: ['doc-1.2'] ..., FPs: ['doc-9.5'] ..., FNs: [] ..., 
at             TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
the            TP: 2, FP: 0, FN: 0, TPs: ['doc-2.2', 'doc-1.1'] ..., FPs: [] ..., FNs: [] ..., 
same           TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
time           TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
in             TP: 1, FP: 0, FN: 0, TPs: ['doc-1.1'] ..., FPs: [] ..., FNs: [] ..., 
meantime       TP: 3, FP: 0, FN: 0, TPs: ['doc-1.1', 'doc-3.5'] ..., FPs: [] ..., FNs: [] ..., 
two            TP: 1, FP: 0, FN: 0, TPs: ['doc-2.1'] ..., FPs: [] ..., FNs: [] ..., 
concurrent     TP: 1, FP: 0, FN: 0, TPs: ['doc-2.1'] ..., FPs: [] ..., FNs: [] ..., 
activities     TP: 1, FP: 0, FN: 0, TPs: ['doc-2.1'] ..., FPs: [] ..., FNs: [] ..., 
are            TP: 1, FP: 0, FN: 0, TPs: ['doc-2.1'] ..., FPs: [] ..., FNs: [] ..., 
triggered      TP: 1, FP: 0, FN: 0

In [111]:
# LITERATURE keywords
print_keyword_dict(literature_and_keyword_metrics, first_x=2, order_by="FP")

in             TP: 1, FP: 2, FN: 0, TPs: ['doc-1.1'] ..., FPs: ['doc-1.4', 'doc-1.4'] ..., FNs: [] ..., 
addition       TP: 0, FP: 2, FN: 0, TPs: [] ..., FPs: ['doc-1.4', 'doc-1.4'] ..., FNs: [] ..., 
to             TP: 0, FP: 2, FN: 0, TPs: [] ..., FPs: ['doc-1.4', 'doc-1.4'] ..., FNs: [] ..., 
whereas        TP: 1, FP: 1, FN: 0, TPs: ['doc-1.2'] ..., FPs: ['doc-9.5'] ..., FNs: [] ..., 
at             TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
the            TP: 2, FP: 0, FN: 0, TPs: ['doc-2.2', 'doc-1.1'] ..., FPs: [] ..., FNs: [] ..., 
same           TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
time           TP: 1, FP: 0, FN: 0, TPs: ['doc-2.2'] ..., FPs: [] ..., FNs: [] ..., 
meantime       TP: 3, FP: 0, FN: 0, TPs: ['doc-1.1', 'doc-3.5'] ..., FPs: [] ..., FNs: [] ..., 
two            TP: 0, FP: 0, FN: 1, TPs: [] ..., FPs: [] ..., FNs: ['doc-2.1'] ..., 
concurrent     TP: 0, FP: 0, FN: 1, TPs: [] ..., FPs: [] ..., FNs: ['doc-2.1'

### d) Combined analysis (counts & FPs)

In [121]:
# available stats data:
goldstandard_xor_gateway_stats
goldstandard_and_gateway_stats
gold_xor_gateway_stats
gold_and_gateway_stats
literature_xor_gateway_stats
literature_and_gateway_stats

# available metrics data
gold_xor_keyword_metrics
gold_and_keyword_metrics
literature_xor_keyword_metrics
literature_and_keyword_metrics

print()

In [127]:
def merge_stats_metrics_dict(metrics, stats):
    merged = {}
    for gateway_token, gateway_token_stats in stats.items():
        gateway_token = gateway_token.lower()
        if gateway_token in metrics:
            merged[gateway_token] = {**gateway_token_stats, **metrics[gateway_token]}
        else:
            merged[gateway_token] = gateway_token_stats
    return merged

gold_xor_merged = merge_stats_metrics_dict(gold_xor_gateway_stats, gold_xor_keyword_metrics)
gold_and_merged = merge_stats_metrics_dict(gold_and_gateway_stats, gold_and_keyword_metrics)
literature_xor_merged = merge_stats_metrics_dict(literature_xor_gateway_stats, literature_xor_keyword_metrics)
literature_and_merged = merge_stats_metrics_dict(literature_and_gateway_stats, literature_and_keyword_metrics)

In [141]:
print_keyword_dict(gold_xor_merged, first_x=0, order_by='FP')

for            TP: 6, FP: 63, FN: 0, count: 59, docs: 23, doc_list: {'doc-8.2', 'doc-10.14', 'doc-8.3', 'doc-1.4', 'doc-6.1', 'doc-3.6', 'doc-9.3', 'doc-6.3', 'doc-2.2', 'doc-6.4', 'doc-9.4', 'doc-5.3', 'doc-10.5', 'doc-5.1', 'doc-1.3', 'doc-9.5', 'doc-3.2', 'doc-9.2', 'doc-2.1', 'doc-1.1', 'doc-3.5', 'doc-5.4', 'doc-4.1'}, 
or             TP: 20, FP: 44, FN: 0, count: 64, docs: 29, doc_list: {'doc-10.14', 'doc-10.2', 'doc-6.1', 'doc-5.2', 'doc-10.12', 'doc-3.6', 'doc-10.3', 'doc-3.8', 'doc-6.3', 'doc-10.6', 'doc-10.11', 'doc-2.2', 'doc-6.4', 'doc-10.8', 'doc-10.7', 'doc-5.3', 'doc-5.1', 'doc-10.10', 'doc-10.1', 'doc-10.4', 'doc-1.3', 'doc-9.5', 'doc-10.13', 'doc-9.2', 'doc-3.3', 'doc-2.1', 'doc-1.1', 'doc-5.4', 'doc-10.9'}, 
if             TP: 61, FP: 6, FN: 0, count: 12, docs: 10, doc_list: {'doc-1.3', 'doc-9.5', 'doc-10.2', 'doc-6.3', 'doc-6.4', 'doc-2.1', 'doc-5.4', 'doc-6.1', 'doc-9.1', 'doc-4.1'}, 
should         TP: 2, FP: 6, FN: 0, count: 6, docs: 3, doc_list: {'doc-9.1', 'doc-